In [1]:
from music21 import *
import tensorflow
import sys
import os
import IPython
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
import keras
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.preprocessing import OneHotEncoder
import random

2023-03-27 13:58:41.977516: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 13:58:42.397762: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-27 13:58:42.397799: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-03-27 13:58:44.044191: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
environment.set('musescoreDirectPNGPath', '../.local/bin/MuseScore-4.0.1.230121751-x86_64.AppImage')

In [88]:
filepath = "pop_songs/"

In [93]:
all_midis= []
names = []
for i in os.listdir(filepath):
    if i.endswith(".mid"):
        tr = filepath + i
        names.append(tr)
        midi = converter.parse(tr)
        all_midis.append(midi)
        

/home/milos/.local/lib/python3.10/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=8, channel=None, data=b'Shawn Mendes, Camila Cabello \x96 Se\xf1orita'>; getting generic Instrument
  warnings.warn(
/home/milos/.local/lib/python3.10/site-packages/music21/midi/translate.py:863: TranslateWarning: Unable to determine instrument from <music21.midi.MidiEvent SEQUENCE_TRACK_NAME, track=7, channel=None, data=b'The Chainsmokers feat. XYL\xd8 - Setting Fires'>; getting generic Instrument
  warnings.warn(


In [95]:
def extract_notes(files):
    notes = []
    pick = None
    for f in files:
        #songs = instrument.partitionByInstrument(f)
        for part in f.parts:
            pick = part.recurse()
            for element in pick:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append(".".join(str(n) for n in element.normalOrder))

    return notes
#Getting the list of notes as Corpus
Corpus= np.array(extract_notes(all_midis))
#print("Total notes in all the Mozzart midis in the dataset:", len(Corpus))

In [96]:
Corpus = np.reshape(Corpus, (len(Corpus), 1))

In [97]:
Corpus.shape

(16523, 1)

In [98]:
Corpus

array([['4.7.11'],
       ['7.11'],
       ['0.4.7'],
       ...,
       ['A1'],
       ['A1'],
       ['A1']], dtype='<U9')

In [99]:
len(list(filter(lambda x : x == 'C3', Corpus)))

168

In [100]:
# Initialize OneHotEncoder object
encoder = OneHotEncoder()

# Fit and transform the data
one_hot_data = encoder.fit_transform(Corpus)

# Convert sparse matrix to array
one_hot_data = one_hot_data.toarray()

# Print the one-hot encoded data
print(one_hot_data)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [101]:
one_hot_data.shape

(16523, 164)

In [102]:
input_len = 5

In [103]:
def getXy(data, step):
    y_ind = np.arange(step, len(data), step)
    y = data[y_ind]
    rows_x = len(y)
    X = data[range(rows_x * step)]
    X = np.reshape(X, (rows_x, step, one_hot_data.shape[1]))
    return X, y

In [104]:
print(len(one_hot_data))
indeksi = np.arange(input_len, len(one_hot_data),input_len)
one_hot_data[indeksi].shape

16523


(3304, 164)

In [105]:
X, y = getXy(one_hot_data, input_len)

In [106]:
X.shape

(3304, 5, 164)

In [107]:
y.shape

(3304, 164)

In [136]:
def create_RNN(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(LSTM(hidden_units, input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.1))
    model.add(LSTM(int(hidden_units/2)))
    model.add(Dense(units=dense_units))
    model.add(Dropout(0.1))
    model.add(Dense(units=one_hot_data.shape[1], activation='softmax'))
    model.compile(loss=tensorflow.losses.CategoricalCrossentropy(), optimizer='adam')
    return model

In [137]:
model = create_RNN(516, 164, input_shape=(input_len, one_hot_data.shape[1]), activation=['softmax', 'softmax'])

In [138]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 5, 516)            1405584   
                                                                 
 dropout_4 (Dropout)         (None, 5, 516)            0         
                                                                 
 lstm_5 (LSTM)               (None, 258)               799800    
                                                                 
 dense_4 (Dense)             (None, 164)               42476     
                                                                 
 dropout_5 (Dropout)         (None, 164)               0         
                                                                 
 dense_5 (Dense)             (None, 164)               27060     
                                                                 
Total params: 2,274,920
Trainable params: 2,274,920
No

In [139]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [140]:
len(np.unique(X))

2

In [142]:
model.fit(X, y, epochs=10)

Epoch 1/10
104/104 [==============================] - 9s 88ms/step - loss: 1.5786
Epoch 2/10
104/104 [==============================] - 9s 89ms/step - loss: 1.4866
Epoch 3/10
104/104 [==============================] - 9s 87ms/step - loss: 1.3979
Epoch 4/10
104/104 [==============================] - 8s 76ms/step - loss: 1.3365
Epoch 5/10
104/104 [==============================] - 8s 76ms/step - loss: 1.2401
Epoch 6/10
104/104 [==============================] - 8s 76ms/step - loss: 1.1967
Epoch 7/10
104/104 [==============================] - 8s 75ms/step - loss: 1.1128
Epoch 8/10
104/104 [==============================] - 8s 76ms/step - loss: 1.0462
Epoch 9/10
104/104 [==============================] - 8s 79ms/step - loss: 0.9725
Epoch 10/10
104/104 [==============================] - 8s 78ms/step - loss: 0.9254


In [143]:
new_corpus = np.array(['A4', '0.4.7', 'E3', 'F3', 'G3'])

In [144]:
new_corpus = np.reshape(new_corpus, (len(new_corpus), 1))

In [145]:
new_corpus.shape

(5, 1)

In [238]:
one_hot_new_data = encoder.transform(new_corpus)
one_hot_new_data = one_hot_new_data.toarray()

In [239]:
one_hot_new_data.shape

(5, 164)

In [240]:
def funkcija(broj):
    if broj > 0.2:
        return 1
    else:
        return 0

In [241]:
s = set({})

In [242]:
1 == 1.0

True

In [243]:
def three_in_a_row(new_note, one_hot_new_data, i):
    for j, _ in enumerate(new_note[0]):
        if new_note[0][j] != one_hot_new_data[i-1][j] or new_note[0][j] != one_hot_new_data[i-2][j]:
            return False
    return True

In [244]:
#izbacio dmol (2,5,9)
def choose_another_note(forbidden):
    c_dur = set({'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', '0.4.7','4.7.11', '5.9.0', '7.11.2', '9.0.4'})
    if np.any(forbidden):
        forbidden = np.reshape(forbidden, (1, -1))
        forbidden = encoder.inverse_transform(forbidden)
        c_dur_valid = c_dur.difference({forbidden[0][0]})
    else:
        c_dur_valid = c_dur
    new_note = np.array([random.choice(list(c_dur_valid))])
    new_note = np.reshape(new_note, (len(new_note), 1))

    new_note = encoder.transform(new_note)
    new_note = new_note.toarray()
    
    return new_note
    
    

In [245]:
def in_scale(new_note):
    c_dur = set({'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', '0.4.7','4.7.11', '5.9.0', '7.11.2', '9.0.4'})
    if encoder.inverse_transform(new_note)[0][0] not in c_dur:
        return False
    return True

In [246]:
one_hot_new_data.shape

(5, 164)

In [247]:
i = input_len

In [248]:
while i < 112:
    new_note = model.predict(np.reshape(one_hot_new_data[i-input_len:i], (1, input_len, one_hot_data.shape[1])))
    new_note = np.array(list(map(funkcija, new_note[0])))
    new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))
    
    while three_in_a_row(new_note, one_hot_new_data, i) or not np.any(new_note[0]) or not in_scale(new_note):
        new_note = choose_another_note(new_note[0])
        new_note = np.reshape(new_note, (1, one_hot_data.shape[1]))
    one_hot_new_data = np.append(one_hot_new_data, new_note, axis=0)
    i += 1



1/1 [==============================] - 0s 29ms/step


In [249]:
one_hot_new_data = encoder.inverse_transform(one_hot_new_data)

In [250]:
one_hot_new_data

array([['A4'],
       ['0.4.7'],
       ['E3'],
       ['F3'],
       ['G3'],
       ['7.11.2'],
       ['G3'],
       ['C3'],
       ['C3'],
       ['E3'],
       ['A3'],
       ['A3'],
       ['A3'],
       ['A3'],
       ['A3'],
       ['A3'],
       ['A3'],
       ['F3'],
       ['A3'],
       ['A3'],
       ['4.7.11'],
       ['A3'],
       ['7.11.2'],
       ['5.9.0'],
       ['G3'],
       ['4.7.11'],
       ['D3'],
       ['9.0.4'],
       ['G3'],
       ['9.0.4'],
       ['9.0.4'],
       ['C3'],
       ['G3'],
       ['7.11.2'],
       ['0.4.7'],
       ['0.4.7'],
       ['9.0.4'],
       ['9.0.4'],
       ['0.4.7'],
       ['5.9.0'],
       ['0.4.7'],
       ['0.4.7'],
       ['0.4.7'],
       ['5.9.0'],
       ['5.9.0'],
       ['F3'],
       ['5.9.0'],
       ['7.11.2'],
       ['7.11.2'],
       ['D3'],
       ['9.0.4'],
       ['9.0.4'],
       ['0.4.7'],
       ['0.4.7'],
       ['0.4.7'],
       ['0.4.7'],
       ['5.9.0'],
       ['5.9.0'],
       ['4.7.11'],
       [

In [251]:
one_hot_new_data.shape

(112, 1)

In [252]:
def chords_n_notes(Snippet):
    Melody = []
    offset = 0 #Incremental
    for i in Snippet:
        #If it is chord
        if ("." in i or i.isdigit()):
            chord_notes = i.split(".") #Seperating the notes in chord
            notes = [] 
            for j in chord_notes:
                inst_note=int(j)
                note_snip = note.Note(inst_note)            
                notes.append(note_snip)
                chord_snip = chord.Chord(notes)
                chord_snip.offset = offset
                Melody.append(chord_snip)
        # pattern is a note
        else: 
            note_snip = note.Note(i)
            note_snip.offset = offset
            Melody.append(note_snip)
        # increase offset each iteration so that notes do not stack
        offset += 1
    Melody_midi = stream.Stream(Melody)   
    return Melody_midi

In [253]:
Melody = chords_n_notes(one_hot_new_data.reshape(112, ))

In [254]:
Melody_midy = stream.Stream(Melody)

In [255]:
Melody.write('midi','Melody_Generated_Better1.mid')

'Melody_Generated_Better1.mid'

In [212]:
print("Sample Audio From Data")
IPython.display.Audio("Melody_Generated.mid") 


Sample Audio From Data


ValueError: rate must be specified when data is a numpy array or list of audio samples.